# Materials
* [Bunch of articles](http://www.mitpressjournals.org/doi/pdf/10.1162/neco.1993.5.6.954) - I strongly recomment this resource, cause it hosts most actual (by year of publishing) articles.
* [realization on C++](https://github.com/BelBES/ESOINN)
* [ESOINN algorithm](http://cs.nju.edu.cn/rinc/SOINN/e-soinn.pdf)
* [Detailed article](http://www.haselab.info/soinn-e.html)

In [1]:
import numpy as np

Define neuron class, which is specific for ESOINN (have specific fields).
* feature_vector - weights
* accamulate_signals – number of signals
* total_points – points $\neq$ number of signals
* density – mean accumulated signals
* subclass_id – mark for subclass

In [12]:
class ESOINN_Neuron:
    def __init__(self, feature_vector=()):
        self.feature_vector = np.array(feature_vector)  
        self.accamulate_signals = 0
        self.total_points = 0
        self.density = 0
        self.subclass_id = -1
    
    def update_accamulate_signals(self, n=1):
        self.accamulate_signals += 1

Define edge class, which contains self existance and age. Edges are stored in adjacency matrix.

*maybe, it can be done without this class?* — **do this with -1:n instead of age and existance**

**use dict of tuples instead of adjacency matrix**

In [3]:
class ESOINN_Edge:
    def __init__(self, exist=0):
        self.exist = exist
        self.age = 0
        
    def inc_age(self, step=1):
        self.age += step

ESOINN Neural Network class.

To start lerning use: `fit()` method, for clasterization use `predict()`.

Hiperparams:
* C1, C2 – coefficents for noise deletion.
* learning_step
* max_age – for edges.

In [10]:
class ESOINN_NN:
    def __init__(self, init_neurons, C1=0.001, C2=1, learning_step=200, max_age=50, 
                 metrics=lambda x,y,axis=1: np.sqrt(np.sum(np.square(np.array(x) - np.array(y)), axis=axis))
                forget=False):
        self.C1 = C1
        self.C2 = C2
        self.learning_step = learning_step
        self.max_age = max_age
        self.signals_amount = 2
        self.metrics = metrics
        self.forget = forget
        
        self.neurons = np.array([
            ESOINN_Neuron(init_neurons[0]),
            ESOINN_Neuron(init_neurons[1])
        ], dtype=ESOINN_Neuron)
        
#         @fixme: use dict of tuples
        self.adjacency_matrix = np.array([
            np.array([
                ESOINN_Edge(),
                ESOINN_Edge()
            ]), 
            np.array([
                ESOINN_Edge(),
                ESOINN_Edge()
            ])
        ], dtype=ESOINN_Edge)
        
        
    
    def fit(self, input_signal):
        self.signals_amount += 1
        
        winners_indexes, distances = self.find_winners(input_signal)
        
        thresholds = [
            self.calc_threshold(input_signal, winners_indexes[0]),
            self.calc_threshold(input_signal, winners_indexes[1])
        ]
        
        if distances[0] > thresholds[0] or distances[1] > thresholds[1]:
            self.create_neuron(input_signal)
            return
        
        self.update_edges_age(winners_indexes[0], 1)
        
        self.build_connection(winners_indexes)
        
        self.update_density(winners_indexes[0])
        
        self.neurons[winners_indexes[0]].update_accamulate_signals()
        
        self.update_feature_vector(winners_indexes[0])
        
        self.remove_old_ages()
        
        if self.signals_amount % self.learning_step == 0:
            self.update_topology()
            
    def predict(self, input_signal):
        pass
    
    def update(self):
        pass
    
    def find_winners(self, input_signal):
        distances = self.metrics(input_signal, [neuron.feature_vector for neuron in self.neurons])
        
        first_winner = float('inf')
        second_winner = float('inf')
        for dist in distances:
            if dist <= first_winner:
                first_winner, second_winner = dist, first_winner
            elif dist < second_winner:
                second_winner = dist
        
        return [list(distances).index(first_winner), list(distances).index(second_winner)], [first_winner, second_winner]
                        
    
    def calc_threshold(self, input_signal, winner_index):
        target_row = self.adjacency_matrix[winner_index]
        if np.array([edge.exist for edge in target_row]).any():
            return np.max([
                self.metrics(self.neurons[winner_index].feature_vector, self.neurons[edge_index].feature_vector, axis=0) 
                for edge_index, edge in enumerate(target_row) 
                if edge.exist
            ])
        else:
            return self.find_winners(self.neurons[winner_index].feature_vector)[1][1]  # 'cause first winner is always current node
    
    def create_neuron(self, input_signal):
        
        self.neurons = np.append(self.neurons, ESOINN_Neuron(input_signal))
        
#         @fixThisFuckingBullshit
        self.adjacency_matrix = np.hstack([
            self.adjacency_matrix, 
            [[ESOINN_Edge()] for i in range(self.adjacency_matrix.shape[0])]
        ])
        self.adjacency_matrix = np.vstack([
            self.adjacency_matrix, 
            [ESOINN_Edge() for i in range(self.adjacency_matrix.shape[1])]  # use here updated form of matrix
        ])
    
    def update_edges_age(self, neuron_index, step=1):
        for edge in self.adjacency_matrix[neuron_index]:
            if edge.exist:
                edge.inc_age(step)
    
    # algorithm 3.2
    def build_connection(self, winners_indexes):
        # case 1
        for neuron_index in winners_indexes:
            if self.neurons[neuron_index].subclass_id == -1:
                self.create_edge(winners_indexes)
                break
#         @fixme
        if self.neurons[winners_indexes[0]].subclass_id == -1 and self.neurons[winners_indexes[1]].subclass_id == -1:
            self.neurons[winners_indexes[0]].subclass_id = winners_indexes[0]
            self.neurons[winners_indexes[1]].subclass_id = winners_indexes[0]
            return
        elif self.neurons[winners_indexes[0]].subclass_id == -1:
            self.neurons[winners_indexes[0]].subclass_id = winners_indexes[1]
            return
        elif self.neurons[winners_indexes[1]].subclass_id == -1:
            self.neurons[winners_indexes[1]].subclass_id = winners_indexes[0]
            return
        
        # case 2
        if self.neurons[winners_indexes[0]].subclass_id == self.neurons[winners_indexes[1]].subclass_id:
            self.create_neuron(winners_indexes)
            return
        
        # case 3
        pass
                
#     @fixme
    def create_edge(self, indexes):
        self.adjacency_matrix[indexes[0]][indexes[1]].exist = 1
        self.adjacency_matrix[indexes[0]][indexes[1]].age = 0
        self.adjacency_matrix[indexes[1]][indexes[0]].exist = 1
        self.adjacency_matrix[indexes[1]][indexes[0]].age = 0
        
    def set_subclass(self, indexes):
        pass    
    
    def update_density(self, neuron_index):
        pass
    
    def update_feature_vector(self, neuron_index):
        pass
    
    def remove_old_ages(self):
        pass
    
    def current_state(self):
        return {
            'signals_amount': self.signals_amount,
            'C1': self.C1,
            'C2': self.C2,
            'lambda': self.learning_step,
            'max_age': self.max_age,
            'metrics': self.metrics,  # name?
            'neurons': self.neurons.copy(),  # think about it
            'adjacency_matrix': [[edge.age if edge.exist else -1 for edge in row] for row in self.adjacency_matrix]
        }

SyntaxError: invalid syntax (<ipython-input-10-215becba4758>, line 4)

## tests

In [8]:
input_signal = np.array([2, 2])
nn = ESOINN_NN([[1, 2], [5, 2]])

nn.fit(input_signal)

for i in range(nn.neurons.shape[0]):
    print(nn.neurons[i].feature_vector)

# nn.current_state()

[1 2]
[5 2]
